### Import bibliothèques

In [14]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [15]:
import os


def lister_fichiers(repertoire):
    return os.listdir(repertoire)

#contient les images dans lesquelles on souhaite faire la recherche
liste_images=lister_fichiers("IMAGES/Images")
liste_images.sort()

#imagettes contenant les caractéristiques à chercher dans les images
liste_avions=lister_fichiers('IMAGES/Imagettes/avions')
liste_drones=lister_fichiers('IMAGES/Imagettes/drones')
liste_helicopter=lister_fichiers('IMAGES/Imagettes/hélicoptères')
liste_missiles=lister_fichiers('IMAGES/Imagettes/missiles')
liste_oiseaux=lister_fichiers('IMAGES/Imagettes/oiseaux')


 ### Fonction de comparaison ORB :

In [16]:
def comparaisonORB (path1, path2):
    img1 = cv2.imread(path1, cv2.IMREAD_GRAYSCALE)
    
    img2 = cv2.imread(path2, cv2.IMREAD_GRAYSCALE)

    orb = cv2.ORB_create()
    points1, descripteursImg1 = orb.detectAndCompute(img1, None)
    points2, descripteursImg2 = orb.detectAndCompute(img2, None)
    
    algoBF = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    paires_corresp = algoBF.knnMatch(descripteursImg1, descripteursImg2, k=5)

    Matches_tri = sorted(paires_corresp, key=lambda x:x[0].distance)

    Matches_tri = Matches_tri[:10]
    
    sumDistances = 0

    for group in Matches_tri:
        for match in group :
            sumDistances += match.distance

    return sumDistances

    
    

 ### Pourcentage de réussite avec ORB

In [ ]:
nb_true=0
nb_false=0


for i in range(0,50):   # Parcours de toutes nos images a tester

    DicoComparaison = {"Avions" : [], "Drones" : [], "Hélicoptères" : [], "Missiles" : [], "Oiseaux" : []} # initialisation des doctionnaires vides
    pathUser = "IMAGES/Images/" + liste_images[i]

    for i in range(10): # Comparaison avec les imagettes
        pathAvions = 'IMAGES/Imagettes/avions/' + liste_avions[i]
        nb=comparaisonORB(pathUser, pathAvions)
        DicoComparaison["Avions"].append(nb)
        pathDrones = 'IMAGES/Imagettes/drones/' + liste_drones[i]
        nb=comparaisonORB(pathUser, pathDrones)
        DicoComparaison["Drones"].append(nb)
        pathHelico = 'IMAGES/Imagettes/hélicoptères/' + liste_helicopter[i]
        nb=comparaisonORB(pathUser, pathHelico)
        DicoComparaison["Hélicoptères"].append(nb)
        pathMissiles = 'IMAGES/Imagettes/missiles/' + liste_missiles[i]
        nb=comparaisonORB(pathUser, pathMissiles)
        DicoComparaison["Missiles"].append(nb)
        pathOiseaux = 'IMAGES/Imagettes/oiseaux/' + liste_oiseaux[i]
        nb=comparaisonORB(pathUser, pathOiseaux)
        DicoComparaison["Oiseaux"].append(nb)

    # Sommes des distances
    sumAvions = np.sum(DicoComparaison["Avions"])
    sumOiseaux = np.sum(DicoComparaison["Oiseaux"])
    sumDrones = np.sum(DicoComparaison["Drones"])
    sumHelicoptere = np.sum(DicoComparaison["Hélicoptères"])
    sumMissiles = np.sum(DicoComparaison["Missiles"])

    sums = {
        "sumAvions": sumAvions,
        "sumDrones": sumDrones,
        "sumMissiles": sumMissiles,
        "sumHelicoptere": sumHelicoptere,
        "sumOiseaux": sumOiseaux,
    }

    # Trouvez le nom de la variable avec la valeur minimale (la plus petite étant censé être la plus ressemblante)
    minVar = min(sums, key=sums.get)
    minSum = sums[minVar]
    
    #test de conformité : étant donné que les images sont triées par noms, les 9 premières sont des avions, etc...
    if 0<=i<=9:
        if minVar == "sumAvions":
            nb_true+=1
        else:
            nb_false+=1

    elif 10<=i<=19:
        if minVar == "sumDrones":
            nb_true+=1
        else:
            nb_false+=1

    elif 20<=i<=29:
        if minVar == "sumHelicoptere":
            nb_true+=1
        else:
            nb_false+=1

    elif 30<=i<=39:
        if minVar == "sumMissiles":
            nb_true+=1
        else:
            nb_false+=1

    elif 40<=i<=49:
        if minVar == "sumOiseaux":
            nb_true+=1
        else:
            nb_false+=1

print("L'algorithme ORB obtient",round((nb_true/50)*100), "% de réussite en prenant en compte la distance comme critère." )

L'algorithme ORB obtient  58  % de réussite en prenant en compte la distance comme critère.
